<a href="https://colab.research.google.com/github/cielo229/AI/blob/main/langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##安裝需要的套件
* langchain：基本的langchain套件
* openai：基本的openai套件
* unstructured：讀取文字檔格式的套件
* chromadb：向量儲存資料庫
* tiktoken套件：OpenAI算token數的套件

In [1]:
!pip install langchain
# !pip install openai
!pip install langchain-openai
!pip install unstructured
!pip install chromadb
!pip install tiktoken
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

##將環境變數讀入

In [ ]:
# 導入 ColabSecrets 用戶資料模組
from google.colab import userdata

# 設置 OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

##先套用OpenAI的API
使用langchain中的OpenAI套件載入大型語言模型，載入OpenAi模型，並且設定最大輸出長度為1024。此部分會收費

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.3,
    max_tokens=512,
    )

##測試沒有RAG時候的問答

In [ ]:
llm.invoke("工專時期第3任校長是誰?")

In [ ]:
llm.invoke("明新科技大學的校訓是什麼?")

In [ ]:
!wget https://raw.githubusercontent.com/zzxx666413/LLM_RAG20240510/master/must.txt

In [ ]:
!wget https://raw.githubusercontent.com/zzxx666413/LLM_RAG20240510/master/2028%E7%B8%BD%E7%B5%B1%E5%80%99%E9%81%B8%E4%BA%BA.txt

##建立本機知識庫QA機器人
Document loaders

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader

# 載入資料夾中所有TXT檔案
loader = DirectoryLoader('/content/', glob='**/*.txt')

# 將資料轉成document物佚，每個檔案會為作為一個document
documents = loader.load()

# 初始化載入器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 物件
embeddings = OpenAIEmbeddings()

# 將 document 透過 openai 的 embeddings 物件計算 embedding向量資料暫時存入 Chroma 向量資料庫用於後續的搜尋
docsearch = Chroma.from_documents(split_docs, embeddings)

# 建立回答物件
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

# 進行回答
result = qa({"query": "工專時期第3任校長是誰?"})
print(result['result'])

In [ ]:
result = qa({"query": "現行明新科技大學之校訓?"})
print(result['result'])

文件分割器的chunk_overlap參數，切分後每個文件裡包含幾個上一個文件結尾的內容，主要作用是為了增加每個文件的上下文關聯。比如chunk_overlap=0時，第一個文件為aaaaaa，第二個為bbbbbb；當chunk_overlap=2時，第一個文件為aaaaaa，第二個為aabbbbbb。

##替模型加入記憶功能
「對話記憶體」（ConversationBufferMemory）用於儲存簡單的對話歷史
ConversationBufferMemory
memory_management

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# 建立記憶體實例，開啟 return_messages 是為了將記憶體指定給 chat模型
# 而 memory_key則是可以讓我們客制我們取得對話記錄時用的 key 值
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 建立 chat 語言模型
# llm_chat = ChatOpenAI()

# 提示設計
prompt_chat = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "你是一個友善的學習助理，你接下來會跟使用者來對話。"
        ),
        # 這裏是一個讓記憶體資料填空的地方。
        # 我們也要設定，使用chat_history 來取得對話記錄
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

conversation_chat = LLMChain(
    llm=llm,
    prompt=prompt_chat,
    verbose=True,
    memory=memory
)

In [ ]:
conversation_chat({
    'question': '你好'
})

In [ ]:
conversation_chat({
    'question': '你可以告訴我英國和美國的首都在哪裡嗎?'
})

In [ ]:
#查詢記憶內容
print("chat_history:", memory.load_memory_variables({}))

##進階記憶功能
ConversationBufferWindowMemory 類別
直譯為「局部窗口對話記憶體」。它的主要功能是限制在一個局部窗口內保存的對話資訊。由於 token 的運算資源有限且需消耗費用，甚至如果語言模型是我們自己架設的，同樣需要大量的運算資源，因此我們不能讓歷史對話資料無窮無盡地累積。

使用ConversationBufferWindowMemory 類別，可以只保存最近的 k 條訊息。

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# 建立 ConversationBufferWindowMemory 實例, k=1 即限制一條訊息
memory_buffer_window = ConversationBufferWindowMemory(k=1)

# 更新上下文資訊
memory_buffer_window.save_context({"input": "你好！"}, {"output": "什麼事？"})
memory_buffer_window.save_context({"input": "今天天氣真好！"}, {"output": "我覺得太熱了！"})
memory_buffer_window.save_context({"input": "這是最新的訊息"}, {"output": "只會記錄這個訊息！"})
# 取得記憶體內儲存的資訊
memory_buffer_window.load_memory_variables({})

#--- 實際的輸出 ---

# {'history': 'Human: 這是最新的訊息\nAI: 只會記錄這個訊息！'}

In [ ]:
# 下方是建立向量資料庫的部分
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import VectorStoreRetrieverMemory

db_chroma = Chroma(embedding_function=OpenAIEmbeddings())

retriever = db_chroma.as_retriever(search_kwargs=dict(k=1))

memory_vs = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)

# 這裏是模擬我們已經有三個對話記錄
memory_vs.save_context({"Human": "我最喜歡的食物是披薩"}, {"AI": "這樣很棒！"})
memory_vs.save_context({"Human": "我最喜歡的運動是游泳"}, {"AI": "很高興你跟我說分享你的嗜好。"})
memory_vs.save_context({"Human": "我不喜歡上班"}, {"AI": "瞭解"})
memory_vs.save_context({"Human": "奇奇自助餐很貴"}, {"AI": "太糟糕了"})

# 使用 load_memory_varialbes 取得使用者問題相似度的歷史資料
print(memory_vs.load_memory_variables({"prompt": "我該看什麼運動節目？"}))
# print(memory_vs.load_memory_variables({"prompt": "昂貴的店家"}))

##整合
Chain類別

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import VectorStoreRetrieverMemory

# 載入資料夾中所有TXT檔案
loader = DirectoryLoader('/content/', glob='**/*.txt')

# 將資料轉成document物佚，每個檔案會為作為一個document
documents = loader.load()

# 初始化載入器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 物件
embeddings = OpenAIEmbeddings()

# 將 document 透過 openai 的 embeddings 物件計算 embedding向量資料暫時存入 Chroma 向量資料庫用於後續的搜尋

docsearch = Chroma.from_documents(split_docs, embeddings)

# 建立檢索器
retriever = docsearch.as_retriever()

# 建立記憶體
memory_vs = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)

# 設置預設的prompt
DEFAULT_TEMPLATE = """
你是一個友善的對話機器人，下面歷史記錄是我們曾經的對話。
Human 是我，AI 是你。請根據歷史記錄中的資訊來回覆我的新問題。

歷史記錄:
{history}

Human：{input}
AI：
"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=DEFAULT_TEMPLATE
)
conversation_with_memory_vs = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory=memory_vs,
    # verbose=True,
    output_key='AI'
)

In [ ]:
conversation_with_memory_vs.predict(input="2028總統候選人有誰?")

In [ ]:
conversation_with_memory_vs.predict(input="我的名字叫做Kevin，很高興認識你")

In [ ]:
conversation_with_memory_vs.predict(input="你還記得我叫什麼名字嗎?")